In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
subs_csi_train = pd.read_csv('../input/subs_csi_train.csv')
subs_csi_test = pd.read_csv('../input/subs_csi_test.csv')
subs_features_train = pd.read_csv('../input/subs_features_train.csv', decimal=b',')
subs_features_test = pd.read_csv('../input/subs_features_test.csv', decimal=b',')
subs_bs_consumption_train = pd.read_csv('../input/subs_bs_consumption_train.csv', decimal=b',')
subs_bs_consumption_test = pd.read_csv('../input/subs_bs_consumption_test.csv', decimal=b',')
# subs_bs_data_session_train = pd.read_csv('../input/subs_bs_data_session_train.csv', decimal=b',')
# subs_bs_data_session_test = pd.read_csv('../input/subs_bs_data_session_test.csv', decimal=b',')
subs_bs_voice_session_train = pd.read_csv('../input/subs_bs_voice_session_train.csv', decimal=b',')
subs_bs_voice_session_test = pd.read_csv('../input/subs_bs_voice_session_test.csv', decimal=b',')

In [ ]:
categorical_columns_features_train = ['SK_ID','COM_CAT#1', 'COM_CAT#3', 'BASE_TYPE', 'ACT', 'ARPU_GROUP', 'COM_CAT#7', 'DEVICE_TYPE_ID', 'INTERNET_TYPE_ID', 'COM_CAT#25', 'COM_CAT#26', 'COM_CAT#34']
print (len(subs_features_train[categorical_columns_features_train]))
cat_cols = []
for col in categorical_columns_features_train:
    cat_cols.append( str((col," : ",len(subs_features_train[col].unique()),'\n')))
cat_cols

In [ ]:
from collections import Counter
get_most_common = lambda values: max(Counter(values).items(), key = lambda x: x[1])[0]
subs_features_train_cat = subs_features_train[categorical_columns_features_train].groupby(['SK_ID']).agg(get_most_common).reset_index()
subs_features_test_cat = subs_features_test[categorical_columns_features_train].groupby(['SK_ID']).agg(get_most_common).reset_index()
subs_features_test_cat.head()

In [ ]:
num_cols = ['SK_ID','COM_CAT#2', 'COM_CAT#8','REVENUE','ITC','VAS','RENT_CHANNEL','ROAM','COST','COM_CAT#17','COM_CAT#18','COM_CAT#19','COM_CAT#20','COM_CAT#21','COM_CAT#22','COM_CAT#23','COM_CAT#27','COM_CAT#28','COM_CAT#29','COM_CAT#30','COM_CAT#31','COM_CAT#32','COM_CAT#33']
subs_features_train_num = subs_features_train[num_cols].groupby('SK_ID').agg('sum').reset_index()
subs_features_test_num = subs_features_test[num_cols].groupby('SK_ID').agg('sum').reset_index()
subs_features_test_num.head()

In [ ]:

import calendar
print (subs_csi_test.CONTACT_DATE.unique(),subs_csi_train.CONTACT_DATE.unique())
days = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
format = lambda x: days[calendar.weekday(2018,int(str(x)[3:]),int(str(x)[0:2]))] if len(str(x))==5 else days[calendar.weekday(2018,int(str(x)[3:]),int('0'+str(x)[0:1]))]
subs_csi_test['DAY'] = subs_csi_test['CONTACT_DATE'].map(format)
subs_csi_train['DAY'] = subs_csi_train['CONTACT_DATE'].map(format)
subs_csi_train.head()

In [ ]:
# categorical_columns_train = [c for c in subs_bs_consumption_train.columns if subs_bs_consumption_train[c].dtype.name == 'object']
# numerical_columns_train = [c for c in subs_bs_consumption_train.columns if subs_bs_consumption_train[c].dtype.name != 'object']
# print (categorical_columns_train)
# print (numerical_columns_train)
subs_bs_consumption_train = subs_bs_consumption_train[['SK_ID','SUM_MINUTES','SUM_DATA_MB','SUM_DATA_MIN']].groupby('SK_ID').agg('mean').reset_index()
subs_bs_consumption_test = subs_bs_consumption_test[['SK_ID','SUM_MINUTES','SUM_DATA_MB','SUM_DATA_MIN']].groupby('SK_ID').agg('mean').reset_index()

subs_bs_voice_session_train = subs_bs_voice_session_train[['SK_ID','VOICE_DUR_MIN']].groupby('SK_ID').agg('mean').reset_index()
subs_bs_voice_session_test = subs_bs_voice_session_test[['SK_ID','VOICE_DUR_MIN']].groupby('SK_ID').agg('mean').reset_index()

In [ ]:
Train = pd.merge(subs_bs_consumption_train, subs_csi_train, on='SK_ID', how='outer')
Train = pd.merge(subs_bs_voice_session_train, Train, on='SK_ID', how='outer')
Train = pd.merge(subs_features_train_num, Train, on='SK_ID', how='outer')
Train = pd.merge(subs_features_train_cat, Train, on='SK_ID', how='outer')
Train = Train.dropna(subset=['SUM_MINUTES'])
Train = Train.dropna(subset=['VOICE_DUR_MIN'])
Train = Train.fillna(0)

Test = pd.merge(subs_csi_test, subs_bs_consumption_test,on='SK_ID', how='outer')
Test = pd.merge(Test, subs_bs_voice_session_test,on='SK_ID', how='outer')
Test = pd.merge(Test, subs_features_test_num,on='SK_ID', how='outer')
Test = pd.merge(Test, subs_features_test_cat,on='SK_ID', how='outer')
Test = Test.fillna(0)
# Test = Test[['SK_ID','CONTACT_DATE','SUM_MINUTES','SUM_DATA_MB','SUM_DATA_MIN','VOICE_DUR_MIN','DAY']]
print (Test.columns)
Test.head()

In [ ]:
cats = [cat for cat in categorical_columns_features_train if cat not in ['SK_ID']]
print (cats)
def OHE(cols,df):
    for col in cols:
        s = df[col].unique()
        print (s)

        # Create a One Hot Dataframe with 1 row for each unique value
        one_hot_df = pd.get_dummies(s, prefix='%s_' % col)
        one_hot_df[col] = s

        print("Adding One Hot values for %s (the column has %d unique values)" % (col, len(s)))
        pre_len = len(df)

        # Merge the one hot columns
        df = df.merge(one_hot_df, on=[col], how="left")
        assert len(df) == pre_len
        print(df.shape)
    return df
Train_Ohe = OHE(cats,Train)
Test_Ohe = OHE(cats,Test)
# Test = OHE("DAY",Test)
print(Train_Ohe.columns,Train_Ohe.shape)
Train_Ohe.head()

In [ ]:
over_col = categorical_columns_features_train
over_col.append('DAY')
over_col.append('CONTACT_DATE')
over_col+=['DEVICE_TYPE_ID__1.0' ,'DEVICE_TYPE_ID__4.0']
print (over_col)
# final_col = [col for col in Test_Ohe.columns if col not in over_col]
# final_col = ['COM_CAT#2', 'COM_CAT#8','REVENUE','ITC','VAS','RENT_CHANNEL','ROAM','COST','COM_CAT#17','COM_CAT#18','COM_CAT#19','COM_CAT#20','COM_CAT#21','COM_CAT#22','COM_CAT#23','COM_CAT#27','COM_CAT#28','COM_CAT#29','COM_CAT#30','COM_CAT#31','COM_CAT#32','COM_CAT#33']
final_col = ['COM_CAT#2', 'COM_CAT#8','REVENUE','ITC','VAS','RENT_CHANNEL','COST','COM_CAT#20','COM_CAT#21','COM_CAT#22','COM_CAT#23','COM_CAT#27','COM_CAT#28','COM_CAT#29','COM_CAT#30','COM_CAT#31','COM_CAT#32','COM_CAT#33']
# final_col = ['COM_CAT#2', 'COM_CAT#8','REVENUE','ITC','VAS','RENT_CHANNEL','COST','COM_CAT#20','COM_CAT#23','COM_CAT#27','COM_CAT#29','COM_CAT#30','COM_CAT#31','COM_CAT#32','COM_CAT#33']


# final_col.append('SK_ID')
final_col, len(final_col)

In [ ]:
# def OHE(col,df):
#     s = df[col].unique()
#     print (s)

#     # Create a One Hot Dataframe with 1 row for each unique value
#     one_hot_df = pd.get_dummies(s, prefix='%s_' % col)
#     one_hot_df[col] = s

#     print("Adding One Hot values for %s (the column has %d unique values)" % (col, len(s)))
#     pre_len = len(df)

#     # Merge the one hot columns
#     df = df.merge(one_hot_df, on=[col], how="left")
#     assert len(df) == pre_len
#     print(df.shape)
#     return df
# Train = OHE("DAY",Train)
# Test = OHE("DAY",Test)
# print(Train.columns,Test.columns)
# Train.head()

In [ ]:
# Train_X = Train[['SUM_MINUTES','SUM_DATA_MB','SUM_DATA_MIN','VOICE_DUR_MIN']].values
# Test_X = Test[['SUM_MINUTES','SUM_DATA_MB','SUM_DATA_MIN','VOICE_DUR_MIN']].values
# Train_X = Train[['SUM_MINUTES','SUM_DATA_MB']].values
# Test_X = Test[['SUM_MINUTES','SUM_DATA_MB']].values
# Train_X = Train[['DAY__Friday', 'DAY__Monday', 'DAY__Saturday', 'DAY__Sunday', 'DAY__Thursday', 'DAY__Tuesday', 'DAY__Wednesday']].values
# Test_X = Test[['DAY__Friday', 'DAY__Monday', 'DAY__Saturday', 'DAY__Sunday', 'DAY__Thursday', 'DAY__Tuesday', 'DAY__Wednesday']].values
# Train_X = Train[['SUM_MINUTES','SUM_DATA_MB','SUM_DATA_MIN','VOICE_DUR_MIN','DAY__Friday', 'DAY__Monday', 'DAY__Saturday', 'DAY__Sunday', 'DAY__Thursday', 'DAY__Tuesday', 'DAY__Wednesday']].values
# Test_X = Test[['SUM_MINUTES','SUM_DATA_MB','SUM_DATA_MIN','VOICE_DUR_MIN','DAY__Friday', 'DAY__Monday', 'DAY__Saturday', 'DAY__Sunday', 'DAY__Thursday', 'DAY__Tuesday', 'DAY__Wednesday']].values

Train_Y = Train_Ohe[['CSI']].values
# Train_X = Train_Ohe[['SUM_MINUTES','SUM_DATA_MB']].values
# Test_X = Test_Ohe[['SUM_MINUTES','SUM_DATA_MB']].values
Train_X = Train_Ohe[final_col].values
Test_X = Test_Ohe[final_col].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Train_X, Train_Y, test_size=0.25, random_state=42)
np.random.seed(0)

from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(256, input_dim=len(final_col),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid')) #sigmoid

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          epochs=24,
          batch_size=16)
score = model.evaluate(X_test, y_test, batch_size=16)
print ('score - ',score)

In [ ]:
import xgboost
from sklearn.metrics import accuracy_score

param = {'max_depth': 50, 'objective': 'binary:logistic','eval_metric': 'auc'}
model2 = xgboost.XGBClassifier(**param)
model2.fit(X_train, y_train)
# make predictions for test data
y_pred = model2.predict(X_test)
# predictions = [round(value) for value in y_pred]
predictions = [value for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
xgboost.plot_importance(model2)

In [ ]:
res1 = model.predict(Test_X, batch_size=16)
res2 = model2.predict(Test_X)
res = np.zeros(5221)
for i in range (len(res1)):
    res[i]=(res1[i]+res2[i])/2
print (res2.max(),res.max(),res1.max())

In [ ]:
sub = pd.DataFrame(res)
sub.to_csv('subm_ens.csv', header=False, index=False)

sub1 = pd.DataFrame(res1)
sub1.to_csv('subm_dens.csv', header=False, index=False)

sub2 = pd.DataFrame(res2)
sub2.to_csv('subm_xgb.csv', header=False, index=False)